# **Bayesian Test 1: Difference Between Weekday and Weekend (Reanalysis of Test 2)**

In [ ]:
!pip install pingouin

In [ ]:
!pip install --upgrade pingouin

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy.stats import ttest_ind
from scipy.stats import pearsonr

In [ ]:
dublin_bikes_cleaned = pd.read_parquet("data/dublin_bikes_cleaned.parquet")

In [ ]:
dublin_bikes_cleaned.head(10)

In [ ]:
# Filtering data using Boolean values for WEEKEND
weekday_vals = dublin_bikes_cleaned[dublin_bikes_cleaned['WEEKEND'] == False]['AVAILABLE BIKES'].dropna()
weekend_vals = dublin_bikes_cleaned[dublin_bikes_cleaned['WEEKEND'] == True]['AVAILABLE BIKES'].dropna()

# Calculate the t-statistic using pg.ttest
t_stat_results = pg.ttest(weekday_vals, weekend_vals, paired=False)
t_stat = t_stat_results['T'].iloc[0] # Extract the t-statistic value


# Run Bayesian independent t-test using the calculated t-statistic
bayes_result = pg.bayesfactor_ttest(t_stat, nx=len(weekday_vals), ny=len(weekend_vals), paired=False)

print(f"Bayes Factor (BF10): {bayes_result:.3f}")

In [ ]:
print(f"Weekday observations: {len(weekday_vals)}")
print(f"Weekend observations: {len(weekend_vals)}")


In [ ]:
# Unpaired t-test (just for stats, even though assumptions weren't met)
t_stat, p_val = ttest_ind(weekday_vals, weekend_vals, equal_var=False)
print(f"T-statistic = {t_stat:.2f}, p-value = {p_val:.5f}")

# Calculate Cohen's d (effect size)
import numpy as np
pooled_std = np.sqrt((np.std(weekday_vals, ddof=1) ** 2 + np.std(weekend_vals, ddof=1) ** 2) / 2)
cohen_d = (np.mean(weekday_vals) - np.mean(weekend_vals)) / pooled_std
print(f"Cohen’s d = {cohen_d:.3f}")

In [ ]:
# plot to check for extreme values
plt.figure(figsize=(8, 4))
sns.boxplot(data=[weekday_vals, weekend_vals])
plt.xticks([0, 1], ['Weekday', 'Weekend'])
plt.title("Boxplot of Available Bikes by Day Type")
plt.show()

To complement the non-parametric Mann-Whitney U test, a Bayesian t-test was performed using Pingouin. This test compared average bike availability between weekdays and weekends across over 2.7 million records.

**Results:**
- Bayes Factor (BF10) ≈ 3.99 × 10²⁷
- T-statistic = –17.75
- Cohen’s d = –0.023 (very small effect size)

**Conclusion:**  
The Bayes Factor indicates **extremely strong statistical evidence** that a difference exists in bike availability between weekdays and weekends. However, the **practical difference is negligible**, as shown by the very small effect size and overlapping distributions in the boxplot.

This outcome is typical in large datasets: even tiny differences can become statistically overwhelming.  
If the goal is **statistical analysis**, this result is valid and confirms a consistent difference.  
If the goal is **real-world decision-making** (e.g., adjusting bike inventory by day type), the result is likely **not actionable**.

In short, **statistically valid, but practically minimal**.

# **Bayesian Test 2: Correlation (Test 3 Reanalysis)**

In [ ]:
# Clean subset
subset = dublin_bikes_cleaned[['BIKE STANDS', 'AVAILABLE BIKES']].dropna()
r, _ = pearsonr(subset['BIKE STANDS'], subset['AVAILABLE BIKES'])
n = len(subset)

print(f"Pearson r = {r:.3f}, Sample size = {n}")

In [ ]:
# Simplified Bayes Factor approximation
z = 0.5 * np.log((1 + r) / (1 - r))  # Fisher Z-transform
se = 1 / np.sqrt(n - 3)
BF10 = np.exp((z**2) / (2 * se**2))

print(f"Estimated Bayes Factor (BF10) for correlation = {BF10:.3f}")


To complement the earlier Pearson and Spearman correlation results, a Bayesian reanalysis was performed using a standard approximation method for large datasets.

**Inputs:**
- Pearson correlation: *r* = 0.215
- Sample size: ~2.78 million

**Estimated Bayes Factor (BF10)**: ∞ (infinite, overflow)

**Conclusion:**  
Due to the massive sample size, even a modest correlation of *r = 0.215* results in an extremely high Bayes Factor. Statistically, this confirms **very strong evidence** for a positive relationship between station size and availability.

However, in **practical terms**, the effect is moderate — station size does influence availability, but it's not the only factor. The result is reliable and stable, but should not be overstated in operational impact.


In [ ]:
avg_by_size = subset.groupby('BIKE STANDS')['AVAILABLE BIKES'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(data=avg_by_size, x='BIKE STANDS', y='AVAILABLE BIKES', marker='o')
plt.title("Average Available Bikes by Station Size")
plt.xlabel("Bike Stands")
plt.ylabel("Average Available Bikes")
plt.grid(True)
plt.show()

#**Final Conclusion**

This statistical mini-project analyzed DublinBikes usage data using a variety of classical and human-centered statistical methods. Our goal was to investigate what factors influence bike availability across time and space in Dublin city.

---

## Summary of Findings

| Test | Question | Method | Result |
|------|----------|--------|--------|
| Test 1 | Does availability vary by hour of day? | Kruskal-Wallis (ANOVA alternative) | Significant difference |
| Test 2 | Weekday vs Weekend? | Mann-Whitney U + Bootstrap + Bayesian t-test | Statistically significant, practically small |
| Test 3 | Does station size correlate with availability? | Pearson + Spearman | Weak but significant correlation |
| Test 4 | Differences between stations? | Kruskal-Wallis | Strong station-level differences |
| Test 5 | Interaction: Hour × Weekend? | Factorial ANOVA | Hour matters, no interaction with weekend |
| Bootstrap | Repeated Test 2 | Confirms stability of small difference |
| Bayesian 1 | Weekday vs Weekend | Extreme statistical evidence, negligible effect size |
| Bayesian 2 | Station size vs availability | Overwhelming Bayesian evidence for weak correlation |

---

## Interpretation & Takeaways

- **Time of day and location are strong predictors** of bike availability.
- **Weekends show a statistically different pattern**, but the practical difference is very small.
- **Larger stations** tend to have more bikes available, but the correlation is weak.
- **Bayesian analysis confirms** frequentist results but also highlights the importance of considering effect size and not just p-values or Bayes Factors.

---

## Human-Centered Consideration

This analysis demonstrates how **statistical significance does not always mean practical importance**. In a high-frequency dataset like this, even very small differences can show extreme certainty (e.g., BF10 → ∞), but decision-makers should consider whether such differences warrant real-world changes.

---

## Final Note

All tests followed a transparent, explainable approach: assumptions were tested, alternatives were justified, and visualizations were used to support interpretations — fulfilling both the statistical and human-centered requirements of the assignment.